In [8]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input
import matplotlib.pyplot as plt
import pandas as pd
import os

# Configuration
MODEL_PATH = "model.h5"
IMAGE_PATHS = ["data/images/pug_145.jpg", "data/images/english_setter_111.jpg", "data/images/american_bulldog_183.jpg"]
INPUT_SIZE = (128, 128)
PATCH_SIZE = 20

# Add occlusion colors to test
OCCLUSION_COLORS = {
    'black': (0, 0, 0),
    'gray': (128, 128, 128),
    'white': (255, 255, 255)
}

# Feature coordinates added by hand
FEATURE_COORDINATES = {
    "pug_145.jpg": {
        'right_eye': (60, 40),
        'left_eye': (100, 50),
        'nose': (80, 55)
    },
    "english_setter_111.jpg": {
        'right_eye': (65, 55),
        'left_eye': (80, 45),
        'nose': (95, 60)
    },
    "american_bulldog_183.jpg": {
        'right_eye': (55, 65),
        'left_eye': (70, 60),
        'nose': (70, 75)
    }
}

def load_and_preprocess_image(image_path):
    """Load and preprocess an image."""
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found at path: {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, INPUT_SIZE)
    return np.expand_dims(img.astype("float32") / 255.0, axis=0)

def create_feature_extractor(model):
    """Create a model that outputs intermediate layer activations."""
    inputs = Input(shape=(INPUT_SIZE[0], INPUT_SIZE[1], 3))
    x = inputs
    outputs = []
    for i, layer in enumerate(model.layers):
        x = layer(x)
        if i in [5, 7]:
            outputs.append(x)
    return Model(inputs=inputs, outputs=outputs)

def compute_occlusion_effect(model, feature_extractor, image, coords, color):
    """Compute the effect of occluding a specific region with given color."""
    occluded = (image[0] * 255).astype(np.uint8)
    x, y = coords
    x_start = max(0, x - PATCH_SIZE//2)
    y_start = max(0, y - PATCH_SIZE//2)
    cv2.rectangle(occluded,
                 (x_start, y_start),
                 (x_start + PATCH_SIZE, y_start + PATCH_SIZE),
                 color, -1)
    occluded_input = np.expand_dims(occluded.astype(np.float32)/255.0, axis=0)

    original_activations = feature_extractor.predict(image, verbose=0)
    occluded_activations = feature_extractor.predict(occluded_input, verbose=0)

    changes = []
    for orig, occ in zip(original_activations, occluded_activations):
        rel_change = np.abs(occ - orig) / (np.abs(orig) + 1e-7)
        changes.append(np.mean(rel_change))

    return changes[0], changes[1], occluded

def compute_random_occlusions(model, feature_extractor, image, color, n_random=5):
    """Compute occlusion effects for random locations."""
    H, W = image.shape[1:3]
    results = []
    occluded_images = []

    for _ in range(n_random):
        x = np.random.randint(PATCH_SIZE, W - PATCH_SIZE)
        y = np.random.randint(PATCH_SIZE, H - PATCH_SIZE)
        l5, l7, occluded = compute_occlusion_effect(model, feature_extractor, image, (x, y), color)
        results.append((l5, l7))
        occluded_images.append(occluded)

    layer5_stats = np.mean([r[0] for r in results]), np.std([r[0] for r in results])
    layer7_stats = np.mean([r[1] for r in results]), np.std([r[1] for r in results])

    return layer5_stats, layer7_stats, occluded_images

def visualize_occlusions(original_image, feature_occlusions, random_occlusions, filename, color_name):
    """Create visualization grid of occlusion effects."""
    fig, axes = plt.subplots(1, 9, figsize=(20, 4))
    fig.suptitle(f'Occlusion Analysis - {color_name} occlusion', y=1.05)

    axes[0].imshow(original_image)
    axes[0].set_title('Original')
    axes[0].axis('off')

    for i, (title, img) in enumerate(zip(['Right Eye', 'Left Eye', 'Nose'], feature_occlusions)):
        axes[i+1].imshow(img)
        axes[i+1].set_title(title)
        axes[i+1].axis('off')

    for i, img in enumerate(random_occlusions[:5]):
        axes[i+4].imshow(img)
        axes[i+4].set_title(f'Random {i+1}')
        axes[i+4].axis('off')

    plt.tight_layout()
    plt.savefig(f'occlusion_grid_{color_name}_{filename}.png', bbox_inches='tight', dpi=300)
    plt.close()

def main():
    model = load_model(MODEL_PATH)
    feature_extractor = create_feature_extractor(model)

    # Store results for each color
    all_results = {}

    for color_name, color in OCCLUSION_COLORS.items():
        print(f"\nProcessing with {color_name} occlusion")
        results = {feature: {'layer5': [], 'layer7': []}
                  for feature in ['right_eye', 'left_eye', 'nose', 'random']}

        for image_path in IMAGE_PATHS:
            print(f"Processing {image_path}")
            image = load_and_preprocess_image(image_path)
            filename = os.path.basename(image_path)
            coords = FEATURE_COORDINATES[filename]

            feature_occlusions = []
            for feature, feature_coords in coords.items():
                l5, l7, occluded = compute_occlusion_effect(model, feature_extractor, image, feature_coords, color)
                results[feature]['layer5'].append(l5)
                results[feature]['layer7'].append(l7)
                feature_occlusions.append(occluded)

            (l5_mean, l5_std), (l7_mean, l7_std), random_occlusions = compute_random_occlusions(
                model, feature_extractor, image, color)
            results['random']['layer5'].append(l5_mean)
            results['random']['layer7'].append(l7_mean)

            visualize_occlusions(image[0], feature_occlusions, random_occlusions, filename, color_name)

        all_results[color_name] = results

    # Create comprehensive results table
    rows = []
    for color_name in OCCLUSION_COLORS.keys():
        for feature in ['right_eye', 'left_eye', 'nose', 'random']:
            results = all_results[color_name][feature]
            rows.append({
                'Color': color_name.capitalize(),
                'Location': feature.replace('_', ' ').title(),
                'Layer 5': f"{np.mean(results['layer5']):.3f} ± {np.std(results['layer5']):.3f}",
                'Layer 7': f"{np.mean(results['layer7']):.3f} ± {np.std(results['layer7']):.3f}"
            })

    df = pd.DataFrame(rows)
    print("\nOcclusion Sensitivity Results for All Colors:")
    print(df.to_string(index=False))
    df.to_csv('occlusion_results_all_colors.csv', index=False)

if __name__ == "__main__":
    main()


Processing with black occlusion
Processing images/pug_145.jpg
Processing images/english_setter_111.jpg
Processing images/american_bulldog_183.jpg

Processing with gray occlusion
Processing images/pug_145.jpg
Processing images/english_setter_111.jpg
Processing images/american_bulldog_183.jpg

Processing with white occlusion
Processing images/pug_145.jpg
Processing images/english_setter_111.jpg
Processing images/american_bulldog_183.jpg

Occlusion Sensitivity Results for All Colors:
Color  Location       Layer 5       Layer 7
Black Right Eye 0.251 ± 0.038 0.422 ± 0.117
Black  Left Eye 0.457 ± 0.395 0.398 ± 0.027
Black      Nose 0.247 ± 0.112 0.533 ± 0.275
Black    Random 0.220 ± 0.019 0.422 ± 0.052
 Gray Right Eye 0.105 ± 0.026 0.205 ± 0.073
 Gray  Left Eye 0.136 ± 0.030 0.184 ± 0.014
 Gray      Nose 0.166 ± 0.038 0.325 ± 0.020
 Gray    Random 0.187 ± 0.091 0.894 ± 0.978
White Right Eye 0.190 ± 0.096 0.473 ± 0.344
White  Left Eye 0.268 ± 0.050 0.489 ± 0.095
White      Nose 0.332 ± 0.010